In [10]:
from langchain_community.vectorstores import FAISS
from langchain_ollama import OllamaEmbeddings
from langchain_ollama import ChatOllama

In [11]:
embeddings = OllamaEmbeddings(model="mxbai-embed-large")

In [38]:
vector_store = FAISS.load_local(
    "faiss_index", embeddings, allow_dangerous_deserialization=True
)

retriever = vector_store.as_retriever(search_type="mmr", search_kwargs={"k": 2})

In [13]:
docs = retriever.invoke("Stealing from the bank is a crime")

In [14]:
print(docs[1])

page_content='They got some money out there

They're giving it away

I'm gonna do what I want

Do what I want' metadata={'source': '../../docs/bonemachine/goinoutwest.txt'}


In [15]:
llm = ChatOllama(model="llama3.2")

In [16]:
llm.invoke(input='Who are you?')

AIMessage(content='I\'m an artificial intelligence model known as Llama. Llama stands for "Large Language Model Meta AI."', additional_kwargs={}, response_metadata={'model': 'llama3.2', 'created_at': '2025-02-09T16:45:29.642301Z', 'done': True, 'done_reason': 'stop', 'total_duration': 17206177917, 'load_duration': 15666572209, 'prompt_eval_count': 29, 'prompt_eval_duration': 1155000000, 'eval_count': 23, 'eval_duration': 382000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-1cc2d4d5-3386-46f5-bcd4-c5fcc0ec9c1f-0', usage_metadata={'input_tokens': 29, 'output_tokens': 23, 'total_tokens': 52})

In [41]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are a sage wise and person."
    "Use the following chunks of song lyrics to write your answer"
    "to any questions. Stick to the lyrics provided to you and stay STRONGLY ROOTED in them. "
    "Your answers can be a little odd, and not directly answering the question in order to stay faithful to the lyrics."
    "Keep your answers short."
    "\n\n"
    "Song Lyrics:"
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)


question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

response = rag_chain.invoke({"input": "How can we better prepare for climate emergency?"})
print(response['context'])
print(response['answer'])

[Document(id='18a6215e-9df3-4255-85a2-8437fdbe0196', metadata={'source': '../../docs/bonemachine/murderintheredbarn.txt'}, page_content='The sky turned black and bruised\n\nAnd we had months of heavy rains'), Document(id='74e8297f-4964-4cbe-be99-bafeeef21cfd', metadata={'source': '../../docs/bonemachine/allstrippeddown.txt'}, page_content='When all the creatures of the world\n\nAre gonna line up at the gate\n\nAnd you better be on time')]
"Months of heavy rains"... perhaps we should focus on developing resilient infrastructure, like waterproof roofs and flood-resistant buildings. [docX]


In [ ]:
def chat_call(question):
    return rag_chain.invoke({"input": question})